
# C2 — Web Scraping MVP · Tottus (Arroz)

**Objetivo:** Entregar un MVP funcional para la tarea C2 utilizando la categoría **Arroz** de Tottus.  
- Prioriza leer el **HTML local** si existe (más estable para la entrega).  
- Si no existe, intenta la **URL en vivo**.  
- Extrae **título**, **marca**, **presentación/unidad**, **precio** y **URL**.  
- Guarda un **CSV** y muestra una vista previa rápida.

> Si Tottus cambia su HTML, el parser alternativo buscará en los bloques JSON incrustados.


In [1]:

# === Celda 1: Importaciones, Config y Helpers ===
# (Descomenta si usas entorno limpio)
# !pip install -q requests beautifulsoup4 lxml pandas

import os, re, time, random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE_URL = "https://www.tottus.com.pe"
START_URL = "https://www.tottus.com.pe/tottus-pe/lista/CATG16815/Arroz"
LOCAL_HTML = "Arroz grandes ofertas _ Tottus Peru.html"

OUT_DIR = "data"
OUT_CSV = os.path.join(OUT_DIR, "tottus_arroz_mvp.csv")
os.makedirs(OUT_DIR, exist_ok=True)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
    "Accept-Language": "es-PE,es;q=0.9,en;q=0.8",
}

def clean_text(s):
    if s is None:
        return None
    s = re.sub(r"\s+", " ", str(s)).strip()
    return s or None

def normalize_price(s):
    if s is None:
        return None
    m = re.findall(r"[\d\.,]+", str(s))
    if not m:
        return None
    val = m[-1]
    # Heurística: 1.234,56 -> 1234.56 ; 1,234.56 -> 1234.56 ; 1234 -> 1234.0
    if val.count(",") and val.count("."):
        val = val.replace(".", "").replace(",", ".")
    else:
        val = val.replace(",", "")
    try:
        return float(val)
    except:
        return None

def load_html():
    if os.path.exists(LOCAL_HTML) and os.path.getsize(LOCAL_HTML) > 0:
        with open(LOCAL_HTML, "r", encoding="utf-8", errors="ignore") as f:
            return f.read(), "local"
    # fallback a web
    r = requests.get(START_URL, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text, "web"


In [2]:

# === Celda 2: Parseo + Export + Vista previa ===

def parse_products_from_dom(html):
    """Extrae productos desde el DOM estático (estructura 'pod' de Tottus)."""
    soup = BeautifulSoup(html, "lxml")
    items = []
    # Base: anclas de cada "pod"
    anchors = soup.select('a[data-pod][data-key], a.pod-link, a[id^="testId-pod-"]')
    seen = set()
    for a in anchors:
        href = a.get("href")
        url = urljoin(BASE_URL, href) if href else None

        # Buscar dentro del mismo "pod"
        pod = a
        # Asegura subir si el título/precio no está como descendiente directo
        for _ in range(4):
            if pod and pod.name != "body" and not pod.find(class_=re.compile(r"pod-details|pod-summary")):
                pod = pod.parent
            else:
                break
        if pod is None:
            pod = a

        # Título del producto (subtítulo largo)
        title_el = pod.select_one('b[id^="testId-pod-displaySubTitle"], .pod-subTitle, .pod-subTitle')
        if not title_el:
            title_el = a.select_one('b[id^="testId-pod-displaySubTitle"], .pod-subTitle')
        titulo = clean_text(title_el.get_text(strip=True) if title_el else None)

        # Marca
        brand_el = pod.select_one('.pod-title')
        marca = clean_text(brand_el.get_text(strip=True) if brand_el else None)

        # Presentación / unidad
        unit_el = pod.select_one('.pod-subtitle-unit')
        presentacion = clean_text(unit_el.get_text(strip=True) if unit_el else None)

        # Precio (atributo data-internet-price o texto en nodo de precio)
        price_li = pod.select_one('li[data-internet-price]')
        if price_li and price_li.has_attr("data-internet-price"):
            precio = price_li.get("data-internet-price")
        else:
            price_text_el = pod.select_one('.prices, [class*="price"]')
            precio = clean_text(price_text_el.get_text(" ", strip=True) if price_text_el else None)

        key = (titulo, url)
        if any([titulo, marca, precio, url]) and key not in seen:
            seen.add(key)
            items.append({
                "titulo": titulo,
                "marca": marca,
                "presentacion": presentacion,
                "precio": precio,
                "precio_num": normalize_price(precio),
                "detalle_url": url,
                "fuente": "DOM"
            })
    return items

def parse_products_from_jsonlike(html):
    """Fallback: detecta objetos producto en JSON incrustado y extrae campos clave."""
    items = []
    display_iter = [(m.start(), m.group(1)) for m in re.finditer(r'"displayName"\s*:\s*"([^"]+)"', html)]
    for pos, name in display_iter:
        window = html[pos: pos + 2000]  # ventana local
        brand = None
        url = None
        price = None

        m_brand = re.search(r'"brand"\s*:\s*"([^"]+)"', window)
        if m_brand:
            brand = m_brand.group(1)

        m_url = re.search(r'"url"\s*:\s*"([^"]+)"', window)
        if m_url:
            url = m_url.group(1)

        m_price = re.search(r'"price"\s*:\s*\[\s*"?([\d\.,]+)"?\s*\]', window)
        if m_price:
            price = m_price.group(1)

        items.append({
            "titulo": clean_text(name),
            "marca": clean_text(brand),
            "presentacion": None,
            "precio": price,
            "precio_num": normalize_price(price),
            "detalle_url": url,
            "fuente": "JSON"
        })
    # Dedup
    clean = []
    seen = set()
    for it in items:
        key = (it.get("titulo"), it.get("detalle_url"))
        if key not in seen:
            seen.add(key)
            clean.append(it)
    return clean

# Cargar HTML
html, origin = load_html()
print(f"Origen del HTML: {origin}")

# Intento 1: DOM
rows = parse_products_from_dom(html)
print(f"DOM → {len(rows)} productos")

# Intento 2 (fallback): JSON incrustado
if len(rows) == 0:
    rows = parse_products_from_jsonlike(html)
    print(f"Fallback JSON → {len(rows)} productos")

# DataFrame + Export
df = pd.DataFrame(rows).drop_duplicates(subset=["titulo", "detalle_url"])
if df.empty:
    print("⚠️ No se extrajeron productos; no se guardará CSV para evitar errores.")
else:
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ Guardado: {OUT_CSV} (filas: {len(df)})")

# Vista previa
if not df.empty:
    display(df.head(10))
    if df["precio_num"].notna().any():
        print("\nResumen precio_num:")
        display(df["precio_num"].describe())


Origen del HTML: web
DOM → 0 productos
Fallback JSON → 0 productos
⚠️ No se extrajeron productos; no se guardará CSV para evitar errores.
